## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [31]:
import pandas as pd

In [32]:
df=pd.read_csv('train.csv')

In [33]:
df.head()
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [34]:
###Drop Nan Values
df=df.dropna()

In [35]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [36]:
## Get the Dependent features
y=df['label']

In [37]:
X.shape

(18285, 4)

In [38]:
y.shape

(18285,)

In [39]:
import tensorflow as tf

In [40]:
tf.__version__

'2.16.1'

In [41]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [42]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [43]:
messages=X.copy()

In [44]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [45]:
messages.reset_index(inplace=True)

In [46]:
import nltk
import re
from nltk.corpus import stopwords

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hrudaykumarkolla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [49]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [50]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1652, 4600, 284, 2030, 3112, 2902, 953, 1888, 1673, 4897],
 [4823, 4168, 4517, 1762, 3581, 4902, 2549],
 [1775, 1552, 4654, 1344],
 [83, 204, 1671, 35, 1707, 884],
 [4501, 3581, 1050, 1887, 4895, 4622, 3581, 1875, 31, 4898],
 [1293,
  2593,
  3244,
  1422,
  958,
  4130,
  2274,
  3105,
  2817,
  4344,
  1745,
  4589,
  4593,
  1326,
  2549],
 [3314, 1473, 1685, 31, 2584, 3592, 2106, 2059, 1258, 1097, 4260],
 [2047, 2000, 2389, 736, 3708, 3187, 4130, 3195, 1258, 1097, 4260],
 [521, 329, 619, 1919, 1313, 4683, 3489, 2209, 4130, 4101],
 [829, 1145, 401, 2548, 2079, 3408, 3527, 2303],
 [4301, 546, 3779, 2950, 2396, 4721, 4421, 1751, 1624, 1346, 807],
 [35, 1203, 3112, 4683, 4130, 3708],
 [2942, 4923, 102, 2915, 3975, 2856, 4882, 648, 4229],
 [3119, 3678, 3346, 3003, 3519, 1277, 2867, 1258, 1097, 4260],
 [675, 687, 1774, 4551, 4548, 1258, 1097, 4260],
 [4018, 3133, 441, 2242, 1242, 1562, 4410, 3146, 2711, 311],
 [1530, 3013, 4168],
 [1609, 2502, 2008, 2090, 4130, 2116, 1900, 2549],
 [194

### Embedding Representation

In [51]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1888 1673 4897]
 [   0    0    0 ... 3581 4902 2549]
 [   0    0    0 ... 1552 4654 1344]
 ...
 [   0    0    0 ... 1258 1097 4260]
 [   0    0    0 ...  955  497 4163]
 [   0    0    0 ... 3792 2390 1899]]


In [52]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1652,
       4600,  284, 2030, 3112, 2902,  953, 1888, 1673, 4897], dtype=int32)

In [53]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

ValueError: Unrecognized keyword arguments passed to Embedding: {'input_length': 20}

In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

NameError: name 'model' is not defined

### Adding Dropout 

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3140,  279],
       [ 278, 2338]], dtype=int64)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.907705053852527